In [4]:
import os
import xml.etree.ElementTree as ET

DATA_DIR = "train"
out_file = "positives.txt"

count_img = 0
count_box = 0

with open(out_file, "w") as f:
    for file in os.listdir(DATA_DIR):
        if not file.endswith(".xml"):
            continue

        xml_path = os.path.join(DATA_DIR, file)
        tree = ET.parse(xml_path)
        root = tree.getroot()

        # tên ảnh trong XML
        filename = root.find("filename").text
        img_path = os.path.join(DATA_DIR, filename)

        if not os.path.exists(img_path):
            continue

        boxes = []

        for obj in root.findall("object"):
            label = obj.find("name").text.lower()
            if label != "face":   # ⚠️ chỉ train face
                continue

            bnd = obj.find("bndbox")
            xmin = int(bnd.find("xmin").text)
            ymin = int(bnd.find("ymin").text)
            xmax = int(bnd.find("xmax").text)
            ymax = int(bnd.find("ymax").text)

            w = xmax - xmin
            h = ymax - ymin

            if w <= 0 or h <= 0:
                continue

            boxes.append(f"{xmin} {ymin} {w} {h}")
            count_box += 1

        if boxes:
            f.write(img_path + " " + str(len(boxes)) + " " + " ".join(boxes) + "\n")
            count_img += 1

print("DONE")
print("Số ảnh positive:", count_img)
print("Tổng số bounding box:", count_box)
print("Đã tạo file:", out_file)


DONE
Số ảnh positive: 5188
Tổng số bounding box: 40239
Đã tạo file: positives.txt


In [1]:
import cv2
import os
import random
import xml.etree.ElementTree as ET

# ================= CONFIG =================
TRAIN_DIR = "train"          # ảnh + xml
NEG_DIR = "negatives"        # output negative

NEG_PER_IMAGE = 3            # số negative mỗi ảnh
MIN_SIZE = 40                # crop tối thiểu (px)
MAX_TRIES = 50               # số lần thử mỗi negative

os.makedirs(NEG_DIR, exist_ok=True)

# ================= UTIL =================
def load_boxes(xml_path):
    """Load ALL bounding boxes trong XML (an toàn nhất)"""
    tree = ET.parse(xml_path)
    root = tree.getroot()
    boxes = []

    for obj in root.findall("object"):
        bnd = obj.find("bndbox")
        xmin = int(bnd.find("xmin").text)
        ymin = int(bnd.find("ymin").text)
        xmax = int(bnd.find("xmax").text)
        ymax = int(bnd.find("ymax").text)
        boxes.append((xmin, ymin, xmax, ymax))

    return boxes


def iou(boxA, boxB):
    """Intersection over Union"""
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    interW = max(0, xB - xA)
    interH = max(0, yB - yA)
    interArea = interW * interH

    if interArea == 0:
        return 0.0

    areaA = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
    areaB = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])

    return interArea / float(areaA + areaB - interArea)


# ================= MAIN =================
neg_count = 0

for file in os.listdir(TRAIN_DIR):
    if not file.lower().endswith((".jpg", ".png", ".jpeg")):
        continue

    img_path = os.path.join(TRAIN_DIR, file)
    xml_path = os.path.join(TRAIN_DIR, file.rsplit(".", 1)[0] + ".xml")

    if not os.path.exists(xml_path):
        continue

    img = cv2.imread(img_path)
    if img is None:
        continue

    H, W = img.shape[:2]

    boxes = load_boxes(xml_path)
    if len(boxes) == 0:
        continue

    for _ in range(NEG_PER_IMAGE):
        for _ in range(MAX_TRIES):
            cw = random.randint(MIN_SIZE, W // 2)
            ch = random.randint(MIN_SIZE, H // 2)
            cx = random.randint(0, W - cw)
            cy = random.randint(0, H - ch)

            crop_box = (cx, cy, cx + cw, cy + ch)

            # đảm bảo KHÔNG giao với bất kỳ face nào
            valid = True
            for fb in boxes:
                if iou(crop_box, fb) > 0.0:
                    valid = False
                    break

            if not valid:
                continue

            crop = img[cy:cy + ch, cx:cx + cw]

            out_name = f"neg_{neg_count:06d}.jpg"
            cv2.imwrite(os.path.join(NEG_DIR, out_name), crop)
            neg_count += 1
            break

print(f"[DONE] Generated {neg_count} negative images.")

# ================= CREATE negatives.txt =================
with open("negatives.txt", "w") as f:
    for img in sorted(os.listdir(NEG_DIR)):
        if img.lower().endswith(".jpg"):
            f.write(os.path.join(NEG_DIR, img) + "\n")

print("[DONE] Created negatives.txt")


[DONE] Generated 15334 negative images.
[DONE] Created negatives.txt


In [2]:
import cv2

fixed = []
bad = 0

with open("positives.txt") as f:
    for line in f:
        parts = line.strip().split()
        img_path = parts[0]
        num = int(parts[1])
        coords = parts[2:]

        img = cv2.imread(img_path)
        if img is None:
            bad += 1
            continue

        H, W = img.shape[:2]
        new_boxes = []

        for i in range(num):
            x = int(coords[i*4])
            y = int(coords[i*4+1])
            w = int(coords[i*4+2])
            h = int(coords[i*4+3])

            # kiểm tra bbox hợp lệ
            if x < 0 or y < 0:
                continue
            if x + w > W or y + h > H:
                continue
            if w < 20 or h < 20:
                continue

            new_boxes.append((x, y, w, h))

        if new_boxes:
            line_new = img_path + " " + str(len(new_boxes))
            for b in new_boxes:
                line_new += f" {b[0]} {b[1]} {b[2]} {b[3]}"
            fixed.append(line_new)

with open("positives_fixed.txt", "w") as f:
    for l in fixed:
        f.write(l + "\n")

print("DONE")
print("Dòng hợp lệ:", len(fixed))
print("Dòng lỗi bị loại:", bad)
print("Đã tạo positives_fixed.txt")


DONE
Dòng hợp lệ: 4646
Dòng lỗi bị loại: 0
Đã tạo positives_fixed.txt
